In [1]:
## get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip

--2020-03-06 04:53:30--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261482368 (249M) [application/zip]
Saving to: ‘concrete_data_week3.zip’

concrete_data_week3 100%[===================>] 249.37M  30.0MB/s    in 8.6s    

2020-03-06 04:53:39 (29.0 MB/s) - ‘concrete_data_week3.zip’ saved [261482368/261482368]



In [0]:
!unzip concrete_data_week3.zip

In [3]:
from keras.preprocessing.image import ImageDataGenerator

import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [0]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

In [0]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [0]:
# using predefined preprocessing
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [7]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [8]:
validation_generator = data_generator.flow_from_directory(
    directory='concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical'
)

Found 10001 images belonging to 2 classes.


In [9]:
model = Sequential()

model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))













94658560/94653016 [==============================] - 2s 0us/step


In [10]:
model.add(Dense(num_classes, activation='softmax'))

In [11]:
model.layers

In [12]:
model.layers[0].layers

In [0]:
# deactive weight for pretrained layer
model.layers[0].trainable = False

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [17]:
fit_history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/2
301/301 [==============================] - 109s 362ms/step - loss: 0.0552 - acc: 0.9820 - val_loss: 0.2052 - val_acc: 0.9140
Epoch 2/2
301/301 [==============================] - 95s 316ms/step - loss: 0.0135 - acc: 0.9964 - val_loss: 0.2123 - val_acc: 0.9175


In [0]:
model.save('classifier_resnet_model.h5')